**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [28]:
import tensorflow

In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization , Flatten

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [3]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [4]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

**Act**: une fonction qui retourne l'action que l'agent doit exécuter pour avancer dans le jeu. Il fera une action similaire au modèle d'apprentissage avec une probabilité 1-epsilon ou une action complétement aléatoire avec une probabilité epsilon.

**Epsilon**: est essentiel pour permettre à l'agent d'explorer l'envirennement de la façon la plus performante et de faire de mieux en mieux des actions plus rentables au fur et à mesure qu'il avance dans le jeu. Si epsilon=0, l'agent va répeter toujours ce qu'il a appris et par conséquence il va rater des opportunités pour maximiser son reward.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [5]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [6]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

L'array **position** est une matrice de zeros avec un 1 dans la ligne et la colonne qui correspondent à la position du rat.

L'array **board** représente l'envirennement et donne l'emplacement des fromages. C'est une matrice de zeros avec 0.5 dans les corddonnées du fromage à manger et -1 dans les corddonnées du fromage empoisoné.


## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.choice(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [9]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
     
    for e in range(epochs):
    # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
    # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
    
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 7.5/11.0. Average score (-3.5)
Win/lose count 7.5/12.0. Average score (-4.0)
Win/lose count 9.0/8.0. Average score (-2.3333333333333335)
Win/lose count 10.5/19.0. Average score (-3.875)
Win/lose count 7.5/11.0. Average score (-3.8)
Win/lose count 9.0/16.0. Average score (-4.333333333333333)
Win/lose count 8.0/11.0. Average score (-4.142857142857143)
Win/lose count 11.5/5.0. Average score (-2.8125)
Win/lose count 8.0/14.0. Average score (-3.1666666666666665)
Win/lose count 6.5/12.0. Average score (-3.4)
Final score: -3.4


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Answer:**

**1-** la fonction $Q^\pi(s,a)$ est définit comme suit:

$$ Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> $$

Elle devient :

$$ Q^\pi(s,a)=E_{p^{\pi}}[ r(s_{0},a_{0}) + \sum_{t \geq 1}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> $$
Par la suite:

$$ Q^\pi(s,a)=E_{p^{\pi}}[ r(s,a) + \gamma \sum_{t \geq 1}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> $$

Comme  $ (s_{t},a_{t}) = (s_{t}^\prime,a_{t}^\prime) $ et $ (s^\prime,a^\prime) = (s_1 , a_1)$, on obtient:

$$ Q^\pi(s,a)=E_{(s',a')\sim p(.|s,a)}[ r(s,a) + E_{p^{\pi}}[ \gamma \sum_{t}\gamma^{t}r(s_{t}^\prime,a_{t}^\prime)|s_{0}^\prime=s^\prime,a_{0}^\prime=a^\prime] ] \> $$

Et finalement: 

$$ Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] $$


**2-** La relation entre la stratégie optimale et la fonction $Q^\pi(s,a)$ est la suivante: 

 $$    \pi^*(s) = \max_a Q^*(s,a)              \                    (1) $$

En utilisant le résultat de la question 1, on obtient:

$$ Q^{*}(s,a)=E_{(s',a')\sim \pi^* (.|s,a)}[r(s,a)+\gamma Q^{\pi^*}(s',\pi^*(s'))] $$

On remplace $ \pi^*(s) $ par son expression en (1):

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

**3-** On a d'après le résultat de la question 2:

\begin{equation*}
E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q^*(s',a',\theta)-Q^*(s,a,\theta)\Vert^{2} = 0 \text{ si Q = Q*  et c'est strictement positive dans le cas échéant}
\end{equation*}

On obtient l'objectif suivant qu'il faut minimiser au cours du training du DQN:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [11]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory)>self.max_memory:
            del self.memory[0]

    def random_access(self):
        idx=np.random.choice(len(self.memory))
        return self.memory[idx]
        

***
The pipeline we will use for training is given below:

In [12]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [13]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        pass

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            transit=self.memory.random_access()
            input_states[i]=transit[0]
            target_q[i]=self.model.predict(transit[0].reshape(1 , 5 ,5 ,self.n_state))
            action=transit[2]
            reward=transit[3]
            
            if transit[-1]:
                target_q[i][action]=reward
            else:
                target_q[i][action]=reward+self.discount*np.max(self.model.predict(transit[1].reshape(1 , 5 ,5 ,self.n_state)))
                
        target_q = np.clip(target_q, -1.5, 1.5)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        model = Sequential()
        model.add(Flatten(input_shape = (5 , 5 , self.n_state)))
        model.add(Dense(128))
        model.add(Activation('relu'))
        model.add(Dense(32))
        model.add(Activation('relu'))
        model.add(Dense(4))

        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train40.mp4'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 000/050 | Loss 0.0113 | Win/lose count 4.5/1.0 (3.5)
Epoch 001/050 | Loss 0.0232 | Win/lose count 3.0/7.0 (-4.0)
Epoch 002/050 | Loss 0.0022 | Win/lose count 1.0/3.0 (-2.0)
Epoch 003/050 | Loss 0.0021 | Win/lose count 1.5/5.0 (-3.5)
Epoch 004/050 | Loss 0.0037 | Win/lose count 4.0/3.0 (1.0)
Epoch 005/050 | Loss 0.0033 | Win/lose count 1.5/5.0 (-3.5)
Epoch 006/050 | Loss 0.0574 | Win/lose count 0/3.0 (-3.0)
Epoch 007/050 | Loss 0.0122 | Win/lose count 2.0/1.0 (1.0)
Epoch 008/050 | Loss 0.0077 | Win/lose count 0.5/2.0 (-1.5)
Epoch 009/050 | Loss 0.0018 | Win/lose count 0.5/4.0 (-3.5)
Epoch 010/050 | Loss 0.0084 | Win/lose count 1.5/0 (1.5)
Epoch 011/050 | Loss 0.0030 | Win/lose count 3.0/2.0 (1.0)
Epoch 012/050 | Loss 0.0071 | Win/lose count 2.0/0 (2.0)
Epoch 013/050 | Loss 0.0303 | Win/lose count 0.5/3.0 (-2.5)
Epoch 014/050 | Loss 0.0013 | Win/lose count 1.5/6.0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [15]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
    
        model=Sequential()
        model.add(Conv2D(64 , 2 , input_shape = (5,5,self.n_state) , padding= 'same'))
        model.add(Activation('relu'))
        model.add(Conv2D(32 , 2))
        model.add(Activation('relu'))
        model.add(Conv2D(16 , 2))
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dense(4))

        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [16]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train40.mp4'))

Epoch 000/050 | Loss 0.0007 | Win/lose count 1.5/4.0 (-2.5)
Epoch 001/050 | Loss 0.0005 | Win/lose count 1.5/2.0 (-0.5)
Epoch 002/050 | Loss 0.0654 | Win/lose count 0/0 (0)
Epoch 003/050 | Loss 0.0007 | Win/lose count 2.0/1.0 (1.0)
Epoch 004/050 | Loss 0.0009 | Win/lose count 2.0/2.0 (0.0)
Epoch 005/050 | Loss 0.0005 | Win/lose count 1.0/3.0 (-2.0)
Epoch 006/050 | Loss 0.0004 | Win/lose count 1.5/1.0 (0.5)
Epoch 007/050 | Loss 0.0064 | Win/lose count 1.5/3.0 (-1.5)
Epoch 008/050 | Loss 0.0078 | Win/lose count 1.0/1.0 (0.0)
Epoch 009/050 | Loss 0.0003 | Win/lose count 1.0/6.0 (-5.0)
Epoch 010/050 | Loss 0.0020 | Win/lose count 1.5/1.0 (0.5)
Epoch 011/050 | Loss 0.0004 | Win/lose count 1.5/1.0 (0.5)
Epoch 012/050 | Loss 0.0417 | Win/lose count 2.0/2.0 (0.0)
Epoch 013/050 | Loss 0.0412 | Win/lose count 1.0/2.0 (-1.0)
Epoch 014/050 | Loss 0.0482 | Win/lose count 1.5/1.0 (0.5)
Epoch 015/050 | Loss 0.0325 | Win/lose count 1.0/4.0 (-3.0)
Epoch 016/050 | Loss 0.0446 | Win/lose count 0.5/4.0 (-

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [17]:
# Temperature 0.3
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 2.5/1.0. Average score (1.5)
Win/lose count 2.0/2.0. Average score (0.75)
Win/lose count 1.0/2.0. Average score (0.16666666666666666)
Win/lose count 1.5/5.0. Average score (-0.75)
Win/lose count 2.0/3.0. Average score (-0.8)
Win/lose count 2.0/4.0. Average score (-1.0)
Win/lose count 1.5/3.0. Average score (-1.0714285714285714)
Win/lose count 0/2.0. Average score (-1.1875)
Win/lose count 0.5/3.0. Average score (-1.3333333333333333)
Win/lose count 2.5/3.0. Average score (-1.25)
Final score: -1.25
Test of the FC
Win/lose count 1.5/3.0. Average score (-1.5)
Win/lose count 3.0/2.0. Average score (-0.25)
Win/lose count 5.0/2.0. Average score (0.8333333333333334)
Win/lose count 1.5/4.0. Average score (0.0)
Win/lose count 3.5/6.0. Average score (-0.5)
Win/lose count 1.5/2.0. Average score (-0.5)
Win/lose count 0/3.0. Average score (-0.8571428571428571)
Win/lose count 1.5/2.0. Average score (-0.8125)
Win/lose count 2.5/0. Average score (-0.4444444444444444)
Win/l

In [19]:
HTML(display_videos('cnn_test0.mp4'))

In [20]:
HTML(display_videos('fc_test0.mp4'))

Je remarque que le FC est plus performant que le CNN.

In [21]:
# Temperature 0.8
env = Environment(grid_size=size, max_time=T,temperature=0.8)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 4.5/2.0. Average score (2.5)
Win/lose count 3.0/3.0. Average score (1.25)
Win/lose count 1.0/2.0. Average score (0.5)
Win/lose count 4.0/0. Average score (1.375)
Win/lose count 4.5/0. Average score (2.0)
Win/lose count 4.5/2.0. Average score (2.0833333333333335)
Win/lose count 5.0/1.0. Average score (2.357142857142857)
Win/lose count 1.5/2.0. Average score (2.0)
Win/lose count 7.5/2.0. Average score (2.388888888888889)
Win/lose count 3.5/2.0. Average score (2.3)
Final score: 2.3
Test of the FC
Win/lose count 8.5/2.0. Average score (6.5)
Win/lose count 4.5/2.0. Average score (4.5)
Win/lose count 7.0/2.0. Average score (4.666666666666667)
Win/lose count 4.5/0. Average score (4.625)
Win/lose count 2.0/2.0. Average score (3.7)
Win/lose count 6.5/1.0. Average score (4.0)
Win/lose count 4.5/3.0. Average score (3.642857142857143)
Win/lose count 5.5/0. Average score (3.875)
Win/lose count 8.5/4.0. Average score (3.9444444444444446)
Win/lose count 4.5/1.0. Average

In [23]:
HTML(display_videos('cnn_test0.mp4'))

In [24]:
HTML(display_videos('fc_test0.mp4'))

Encore ici, le FC est plus performant que le CNN. De plus, le fait de changer la température permet d'augmenter le score de l'agent. Cela est dû tout simplement au fait que l'augmentation de la température signifie qu'il y a plus de fromage à manger que de fromage empoisonné.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [25]:
def train_explore(agent,env,epoch,prefix=''):
# Number of won games
    score = 0
    loss = 0
    agent.set_epsilon(0.9)
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon*0.95)

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
        


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.malus_position[self.x,self.y]=0.1
        self.t = 0
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [26]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore40.mp4'))

Epoch 000/050 | Loss 0.0016 | Win/lose count 0/15.299999999999962 (-15.299999999999962)
Epoch 001/050 | Loss 0.0011 | Win/lose count 0/13.899999999999967 (-13.899999999999967)
Epoch 002/050 | Loss 0.0008 | Win/lose count 0/13.999999999999966 (-13.999999999999966)
Epoch 003/050 | Loss 0.0009 | Win/lose count 0/15.099999999999962 (-15.099999999999962)
Epoch 004/050 | Loss 0.0004 | Win/lose count 0/16.499999999999964 (-16.499999999999964)
Epoch 005/050 | Loss 0.0005 | Win/lose count 0/14.799999999999963 (-14.799999999999963)
Epoch 006/050 | Loss 0.0004 | Win/lose count 0/16.19999999999996 (-16.19999999999996)
Epoch 007/050 | Loss 0.0006 | Win/lose count 0/15.69999999999996 (-15.69999999999996)
Epoch 008/050 | Loss 0.0008 | Win/lose count 0/14.499999999999964 (-14.499999999999964)
Epoch 009/050 | Loss 0.0008 | Win/lose count 0/16.699999999999967 (-16.699999999999967)
Epoch 010/050 | Loss 0.0005 | Win/lose count 0/16.09999999999996 (-16.09999999999996)
Epoch 011/050 | Loss 0.0003 | Win/lose

In [27]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore0.mp4'))

Win/lose count 0/19.400000000000006. Average score (-19.400000000000006)
Win/lose count 0/19.400000000000006. Average score (-19.400000000000006)
Win/lose count 0/19.0. Average score (-19.26666666666667)
Win/lose count 0/19.60000000000001. Average score (-19.350000000000005)
Win/lose count 0/19.300000000000004. Average score (-19.340000000000003)
Win/lose count 0/19.500000000000007. Average score (-19.36666666666667)
Win/lose count 0/19.400000000000006. Average score (-19.371428571428574)
Win/lose count 0/19.400000000000006. Average score (-19.375000000000004)
Win/lose count 0/19.400000000000006. Average score (-19.37777777777778)
Win/lose count 0/19.70000000000001. Average score (-19.410000000000004)
Final score: -19.410000000000004


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***